<a href="https://colab.research.google.com/github/aaronfarquhar/13006610-dataanylitics/blob/main/Assessment_2_DAOTW_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Introduction**

In this report a Linear Regresion model will be crated for predicting the Number of collisions based on trends derived from the previous report's datasets.

In [45]:
#imports for python
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#read data source and create dataframe
df = pd.read_csv('https://raw.githubusercontent.com/aaronfarquhar/13006610-dataanylitics/main/bq-results-20230421-092737-1682069277611.csv')

In [46]:
print(tf.__version__) #for checking the tensorflow import

2.12.0


**Methodology**

As can be seen below the collated data from assessment 1 has been narrrowed down to the parts that showed clear trends. this is the Tempreture, Visiblity, day, Month, and year. this first part will look at tempreture and build a linear regresion model for it. As a reminder: temp (Mean temperature for the day in degrees Fahrenheit to tenths. Missing = 9999.9), visib (Mean visibility for the day in miles to tenths. Missing = 999.9), day, mo (month), and year. The data is ordered by Year, Month, Day.

In [47]:
#check data is present and succsessfully read
print(df)

      temp  visib  day  mo  year  NUM_COLLISIONS
0     76.0    8.9    1   7  2012             505
1     78.5   10.0    1   7  2012             533
2     79.0    9.2    1   7  2012             528
3     73.1   10.0    1   7  2012             530
4     80.3   10.0    1   7  2012             564
...    ...    ...  ...  ..   ...             ...
3160  51.5    6.3    6  12  2020             278
3161  38.5   10.0    7  12  2020             194
3162  39.3    9.8    7  12  2020             241
3163  53.4    6.3    7  12  2020             265
3164  34.2   10.0    7  12  2020             184

[3165 rows x 6 columns]


In [48]:
#temp and collisions table
inputs_for_training = [df['temp'], df['NUM_COLLISIONS']]
header = ['Mean Temputure', 'Number of Collisions']
combined1 = pd.concat(inputs_for_training, axis = 1, keys=header)
print (combined1)

      Mean Temputure  Number of Collisions
0               76.0                   505
1               78.5                   533
2               79.0                   528
3               73.1                   530
4               80.3                   564
...              ...                   ...
3160            51.5                   278
3161            38.5                   194
3162            39.3                   241
3163            53.4                   265
3164            34.2                   184

[3165 rows x 2 columns]


The Data for training the model will be divide 80/20. where 80% of the data is used to train the model and 20% is used to test the model. this split was chosen as this is a connom split used in most Models and there does not apear to be any factors that may require a different split.

In [49]:
#training and testing data
train_dataset = combined1.sample(frac=0.8, random_state=0)
test_dataset = combined1.drop(train_dataset.index)

#prints for checking to ensure data has split properly
print(train_dataset)
print(test_dataset)


      Mean Temputure  Number of Collisions
2533            48.8                   592
1436            52.5                   638
2567            52.5                   722
3139            40.8                   320
1473            65.7                   742
...              ...                   ...
1181            71.4                   545
337             60.8                   707
46              72.0                   618
747             51.4                   451
2891            46.7                   127

[2532 rows x 2 columns]
      Mean Temputure  Number of Collisions
0               76.0                   505
3               73.1                   530
7               80.3                   574
21              81.9                   638
24              84.9                   490
...              ...                   ...
3142            37.7                   315
3143            34.1                   238
3145            38.5                   360
3153            41.5         

Data is copied for usage, so that the data cannot be altered in any way by mistake, and so that the data can be reused more easally in the future if required without having to restart the entire runtime.

In [50]:
train_copy = train_dataset.copy()
test_copy = test_dataset.copy()

training_label = train_copy.pop('Number of Collisions')
testing_labels = test_copy.pop('Number of Collisions')

print(test_copy)
print(testing_labels)

      Mean Temputure
0               76.0
3               73.1
7               80.3
21              81.9
24              84.9
...              ...
3142            37.7
3143            34.1
3145            38.5
3153            41.5
3159            52.1

[633 rows x 1 columns]
0       505
3       530
7       574
21      638
24      490
       ... 
3142    315
3143    238
3145    360
3153    348
3159    285
Name: Number of Collisions, Length: 633, dtype: int64


Below the model is set up, with a normalisation layer to convert the intput data to a more usable form for the model. it is then added to the model itself and complied for use.

In [51]:
from keras.layers.preprocessing import normalization
normalisation_layer = tf.keras.layers.Normalization(input_shape=[1,], axis = None)
#convert values to -1 to 1
normalisation_layer.adapt(np.array(train_copy))

In [52]:
model=tf.keras.Sequential([
    normalisation_layer,
    tf.keras.layers.Dense(units=1)
])

In [53]:
model.compile(
    optimizer = tf.optimizers.Adam(learning_rate = 0.1),
    loss = 'mean_absolute_error'
    )

**Results**

In [54]:
%%time
model_output = model.fit(train_copy, training_label, epochs=100, validation_split = 0.2, verbose = 1 )

Epoch 1/100
64/64 [==============================] - 1s 5ms/step - loss: 558.0705 - val_loss: 554.5651
Epoch 2/100
64/64 [==============================] - 0s 4ms/step - loss: 551.6661 - val_loss: 548.1686
Epoch 3/100
64/64 [==============================] - 0s 3ms/step - loss: 545.2657 - val_loss: 541.7712
Epoch 4/100
64/64 [==============================] - 0s 3ms/step - loss: 538.8659 - val_loss: 535.3707
Epoch 5/100
64/64 [==============================] - 0s 3ms/step - loss: 532.4642 - val_loss: 528.9695
Epoch 6/100
64/64 [==============================] - 0s 4ms/step - loss: 526.0725 - val_loss: 522.5721
Epoch 7/100
64/64 [==============================] - 0s 4ms/step - loss: 519.6669 - val_loss: 516.1692
Epoch 8/100
64/64 [==============================] - 0s 4ms/step - loss: 513.2705 - val_loss: 509.7733
Epoch 9/100
64/64 [==============================] - 0s 4ms/step - loss: 506.8672 - val_loss: 503.3734
Epoch 10/100
64/64 [==============================] - 0s 2ms/step - loss:

As can be seen above, the model is ran for 100 epochs. at the end of the running it can be seen that the reported Val_loss of the model was around 103. since it cannot be seen that the loss ever began creeping higher again, it can be presumed that the model is reasonably accurate. 

In [55]:
accuracy = model.evaluate(test_copy, testing_labels)

20/20 [==============================] - 0s 1ms/step - loss: 96.0671


In [56]:
print(accuracy)

96.06710052490234


Strangely when the model is ran through the evaluate function to test its accuracy, it's loss value is much lower, pehaps indecating that the model may be able to be improved further.

In [57]:
new_data = np.array([12,3.5,100])

predictions = model.predict(new_data)
print(predictions)

1/1 [==============================] - 0s 48ms/step
[[515.8599 ]
 [507.83664]
 [598.9247 ]]


when new data is added and ran through the model to predict the collisions, there are no glaring issues that can be seen. All predictions are within reasonable bounds based on the rest of the data. This report is continued in part 2.